In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as F
import pandas as pd
import plotly
import plotly.express as px

spark = SparkSession.builder.getOrCreate()

# Reading, selecting columns and changing data types

orders = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("/home/jovyan/work/EPAM_sales_analysis/data/olist_orders_dataset.csv") \
                            .select("order_id","order_delivered_carrier_date","order_estimated_delivery_date")
orders = orders.withColumn('order_delivered_carrier_date',to_date(unix_timestamp \
            (orders.order_delivered_carrier_date, 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))) \
            .withColumn('order_estimated_delivery_date',to_date(unix_timestamp \
            (orders.order_estimated_delivery_date, 'yyyy-MM-dd HH:mm:ss').cast('timestamp')))

# We drop rows with null values because we assume that these orders never will by deliverd
orders = orders.na.drop("any")

# two_days column description
  
# True - order was sent more then 2 days before expected delivery date   
# False - The order has not been shipped more than two days before the expected delivery date

orders_big = orders.withColumn('two_days', F.date_add(orders['order_delivered_carrier_date'],2))
orders_big = orders_big.withColumn("two_days", F.when(F.col("two_days") < F.col("order_estimated_delivery_date"), True) \
                                   .otherwise(False))

two_days = orders_big.groupBy("two_days").count()
two_days.write.parquet("/home/jovyan/work/EPAM_sales_analysis/raport/transformed_data/6_task_two_days.parquet",mode="overwrite")
two_days.show()

22/08/18 17:25:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/18 17:25:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/18 17:25:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
